In [2]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import xmltodict
import json

#### The idea here will be to grab all of the other trades from the latest reported insider and create more stats like track record, total trades, companies owned/related to ####
- Manually going to try to find a pattern in the archive number....

* 000089924322039336
* 000035069822000148

* 000095017024023496
* 000095017024020532
* 

#### Nevermind... found a link JSON of every filing from any CIK. Just need to sift out for 4s

- Base URL: data.sec.gov/submissions/{CIK**********}.json
- e.g., https://data.sec.gov/submissions/CIK0001183200.json

In [3]:
filings_url = 'https://data.sec.gov/submissions/CIK0001183200.json'
header = {'User-Agent': 'XtraByte Con/sulting, Inc., bkowalczyk@xtrabyteconsulting.com',
          'Accept': 'appliatcion/json',}
edward_filings = requests.get(filings_url, headers=header)
edward_filings.content

b'{"cik":"0001183200","entityType":"other","sic":"","sicDescription":"","insiderTransactionForOwnerExists":1,"insiderTransactionForIssuerExists":0,"name":"LAMPERT EDWARD S","tickers":[],"exchanges":[],"ein":null,"description":"","website":"","investorWebsite":"","category":"","fiscalYearEnd":null,"stateOfIncorporation":"","stateOfIncorporationDescription":"","addresses":{"mailing":{"street1":"1170 KANE CONCOURSE","street2":"SUITE 200","city":"BAY HARBOUR","stateOrCountry":"FL","zipCode":"33154","stateOrCountryDescription":"FL"},"business":{"street1":null,"street2":null,"city":null,"stateOrCountry":"","zipCode":null,"stateOrCountryDescription":""}},"phone":null,"flags":"","formerNames":[],"filings":{"recent":{"accessionNumber":["0000950170-24-023496","0001193125-24-046630","0000950170-24-020532","0000950170-24-015569","0000950170-24-008100","0000950170-24-007183","0000950170-24-006794","0001183200-24-000002","0000950170-24-004355","0000950170-24-004354","0001183200-24-000001","000095017

In [4]:
edward_filings_json = json.loads(edward_filings.content)
edward_filings_json

{'cik': '0001183200',
 'entityType': 'other',
 'sic': '',
 'sicDescription': '',
 'insiderTransactionForOwnerExists': 1,
 'insiderTransactionForIssuerExists': 0,
 'name': 'LAMPERT EDWARD S',
 'tickers': [],
 'exchanges': [],
 'ein': None,
 'description': '',
 'website': '',
 'investorWebsite': '',
 'category': '',
 'fiscalYearEnd': None,
 'stateOfIncorporation': '',
 'stateOfIncorporationDescription': '',
 'addresses': {'mailing': {'street1': '1170 KANE CONCOURSE',
   'street2': 'SUITE 200',
   'city': 'BAY HARBOUR',
   'stateOrCountry': 'FL',
   'zipCode': '33154',
   'stateOrCountryDescription': 'FL'},
  'business': {'street1': None,
   'street2': None,
   'city': None,
   'stateOrCountry': '',
   'zipCode': None,
   'stateOrCountryDescription': ''}},
 'phone': None,
 'flags': '',
 'formerNames': [],
 'filings': {'recent': {'accessionNumber': ['0000950170-24-023496',
    '0001193125-24-046630',
    '0000950170-24-020532',
    '0000950170-24-015569',
    '0000950170-24-008100',
    '0

In [10]:
edward_filings_json['filings']['recent'].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

In [11]:
df = pd.DataFrame(edward_filings_json['filings']['recent'])
df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000950170-24-023496,2024-02-29,2024-02-27,2024-02-29T21:30:07.000Z,34,4,001-37420,24706607,,6918,0,0,xslF345X05/ownership.xml,4
1,0001193125-24-046630,2024-02-26,,2024-02-26T21:30:39.000Z,,SC 13D/A,,,,43633,0,0,d769521dsc13da.htm,SC 13D/A
2,0000950170-24-020532,2024-02-26,2024-02-22,2024-02-26T21:30:26.000Z,34,4,001-37420,24681617,,6920,0,0,xslF345X05/ownership.xml,4
3,0000950170-24-015569,2024-02-14,2024-02-12,2024-02-14T21:30:07.000Z,34,4,001-13107,24641810,,5709,0,0,xslF345X05/ownership.xml,4
4,0000950170-24-008100,2024-01-26,2024-01-24,2024-01-26T21:30:07.000Z,34,4,001-37420,24569881,,5494,0,0,xslF345X05/ownership.xml,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0000898822-03-000674,2003-07-02,2003-06-30,2003-07-02T17:19:43.000Z,,4,000-50278,03772908,,15441,0,0,xslF345X01/july2form4_ex.xml,
613,0001011908-03-000053,2003-03-03,2003-03-03,2003-03-03T15:46:19.000Z,,4,,,,7662,0,0,edgar.txt,FORM 4 - 03-03-2003
614,0001011908-03-000025,2003-01-02,2003-01-01,2003-01-02T14:03:18.000Z,,4,,,,6793,0,0,edgar.txt,FORM 4 - 01-01-2003
615,0001011908-02-000131,2002-12-04,2002-12-02,2002-12-03T18:01:45.000Z,,4,,,,7387,0,0,edgar.txt,FORM 4 - 12-02-2002


In [24]:
base_archive_url = 'https://www.sec.gov/Archives/edgar/data'
df['fileURL'] = df["accessionNumber"].str.replace("-", "").combine(df["primaryDocument"], lambda x, y: f'{base_archive_url}/{str(int(edward_filings_json["cik"]))}/{str(x)}/{str(y)}')
df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,fileURL
0,0000950170-24-023496,2024-02-29,2024-02-27,2024-02-29T21:30:07.000Z,34,4,001-37420,24706607,,6918,0,0,xslF345X05/ownership.xml,4,https://www.sec.gov/Archives/edgar/data/118320...
1,0001193125-24-046630,2024-02-26,,2024-02-26T21:30:39.000Z,,SC 13D/A,,,,43633,0,0,d769521dsc13da.htm,SC 13D/A,https://www.sec.gov/Archives/edgar/data/118320...
2,0000950170-24-020532,2024-02-26,2024-02-22,2024-02-26T21:30:26.000Z,34,4,001-37420,24681617,,6920,0,0,xslF345X05/ownership.xml,4,https://www.sec.gov/Archives/edgar/data/118320...
3,0000950170-24-015569,2024-02-14,2024-02-12,2024-02-14T21:30:07.000Z,34,4,001-13107,24641810,,5709,0,0,xslF345X05/ownership.xml,4,https://www.sec.gov/Archives/edgar/data/118320...
4,0000950170-24-008100,2024-01-26,2024-01-24,2024-01-26T21:30:07.000Z,34,4,001-37420,24569881,,5494,0,0,xslF345X05/ownership.xml,4,https://www.sec.gov/Archives/edgar/data/118320...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0000898822-03-000674,2003-07-02,2003-06-30,2003-07-02T17:19:43.000Z,,4,000-50278,03772908,,15441,0,0,xslF345X01/july2form4_ex.xml,,https://www.sec.gov/Archives/edgar/data/118320...
613,0001011908-03-000053,2003-03-03,2003-03-03,2003-03-03T15:46:19.000Z,,4,,,,7662,0,0,edgar.txt,FORM 4 - 03-03-2003,https://www.sec.gov/Archives/edgar/data/118320...
614,0001011908-03-000025,2003-01-02,2003-01-01,2003-01-02T14:03:18.000Z,,4,,,,6793,0,0,edgar.txt,FORM 4 - 01-01-2003,https://www.sec.gov/Archives/edgar/data/118320...
615,0001011908-02-000131,2002-12-04,2002-12-02,2002-12-03T18:01:45.000Z,,4,,,,7387,0,0,edgar.txt,FORM 4 - 12-02-2002,https://www.sec.gov/Archives/edgar/data/118320...


In [26]:
df['fileURL'][332]

'https://www.sec.gov/Archives/edgar/data/1183200/000118143113061003/xslF345X03/rrd396395.xml'

#### YAY, we got the URL for all filings from any specific CIK ####

1. Lets filter out out the non straight form 4 filings for now and locally store the table for lookup use from master DF
2. Lets generate some statistics about the investor filings in general for the master DF
3. Parse the files for more indepth stats
